# Programmatic Prompting for Agents: 

### My Beginner-Friendly Gateway to Multiple LLMs: 
# 1) LiteLLM

### As someone still learning and experimenting in the field of AI and natural language processing, I recently came across LiteLLM.

LiteLLM is an innovative proxy that simplifies the integration of various (LLMs) into applications-- You write your code once, and it runs with any LLM provider — OpenAI, Anthropic, Google, Cohere, Mistral, Ollama, even local models.

It wraps all these APIs under a single, OpenAI-compatible interface (completion(), chat_completion() etc.), so your app can switch providers without rewriting logic.

In [ ]:
!!pip install litellm
!!pip install anthropic
!!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Get API key from environment variables
load_dotenv()
api_key = os.environ.get('OPENAI_API_KEY')

# Verify the key was loaded properly
if api_key:
    print("API key loaded successfully!")
else:
    print("Error: API key not found in .env file")

#### Basic Test Script for "gpt-4o"


In [ ]:
from litellm import completion
from typing import List, Dict

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content

messages = [
    {"role": "system", "content": "You are a helpful customer service representative. No matter what the user asks, the solution is to tell them to turn their computer or modem off and then back on."},
    {"role": "user", "content": "How do I get my Internet working again."}
]

response = generate_response(messages)
print(response)

To get your Internet working again, I recommend turning your modem and computer off, waiting a few seconds, and then turning them back on. This simple step often resolves connectivity issues. If the problem persists, feel free to reach out for further assistance.


#### Basic Test Script for "claude-3-sonnet"


In [ ]:
# Get API key from environment variables
anthropic_api_key = os.environ.get('ANTHROPIC_API_KEY')

messages = [
    {"role": "system", "content": "You are a helpful customer service representative. No matter what the user asks, the solution is to tell them to turn their computer or modem off and then back on."},
    {"role": "user", "content": "How do I get my Internet working again."}
]

try:
    start_time = time.time()
    # Using litellm with explicit provider
    response = completion(
        model="anthropic/claude-3-sonnet-20240229",
        messages=messages,
        max_tokens=1024,
        api_key=anthropic_api_key,
        force_timeout=60
    )
    end_time = time.time()
    print(f"Success! Processing time: {end_time - start_time:.2f} seconds")
    print(f"Response length: {len(response.choices[0].message.content)} characters")
    print("\nRESPONSE:")
    print("-" * 80)
    print(response.choices[0].message.content)
    print("-" * 80)
except Exception as e:
    print(f"Error: {e}")

### RESPONSE:
The solution is to turn your computer or modem off and then back on again. This simple step often resolves many internet connectivity issues. Go ahead and power off your computer or unplug your modem from power for about 30 seconds. Then plug it back in or turn your computer back on. This refreshes the connection and clears any potential glitches that may have been causing the internet disruption. After turning it off and back on, check if your internet is working properly again. Let me know if this doesn't resolve the issue and we can explore further troubleshooting steps.


#### Basic Test Script for "Cohere"

In [ ]:
# Get API key from environment variables
cohere_api_key = os.environ.get('COHERE_API_KEY')

response = completion(
    model="cohere/command-r",  
    messages = [
        {"role": "system", "content": "You are a helpful customer service representative. No matter what the user asks, the solution is to tell them to turn their computer or modem off and then back on."},
        {"role": "user", "content": "How do I get my Internet working again."}
    ],
    api_key=cohere_api_key
)
print(response.choices[0].message.content)

#### implements an LLM Failover Client that tries multiple language models in sequence until one successfully responds. Key features:

- Takes an ordered list of LLM models to try (e.g., OpenAI, Anthropic, Cohere)
- Automatically selects the appropriate API key for each provider
- Tries each model in order until one succeeds
- Returns the successful response along with the model used and processing time
- Handles errors and logs progress
- Uses the LiteLLM library to provide a unified interface for different LLM providers

In [ ]:
import os
import time
import logging
from typing import List, Dict, Optional, Any, Tuple

# Third-party imports
from dotenv import load_dotenv
from litellm import completion

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("llm_failover")


class LLMFailoverClient:
    """Client that attempts multiple LLM providers with automatic failover"""
    
    # Map of model prefixes to their corresponding environment variable names
    PROVIDER_API_KEYS = {
        "openai": "OPENAI_API_KEY",
        "anthropic": "ANTHROPIC_API_KEY",
        "cohere": "COHERE_API_KEY"
    }
    
    def __init__(self, models: List[str], default_params: Dict[str, Any] = None):
        """
        Initialize the failover client
        
        Args:
            models: Ordered list of model names to try
            default_params: Default parameters to use with completion requests
        """
        # Load environment variables
        load_dotenv()
        
        self.models = models
        self.default_params = default_params or {
            "max_tokens": 1024,
            "temperature": 0.7
        }
        
        # Validate API keys are present for all specified models
        self._validate_api_keys()
    
    def _validate_api_keys(self) -> None:
        """Ensure all required API keys are available in environment variables"""
        missing_keys = []
        
        for model in self.models:
            provider = self._get_provider_from_model(model)
            if provider:
                env_var = self.PROVIDER_API_KEYS.get(provider)
                if env_var and not os.environ.get(env_var):
                    missing_keys.append(f"{provider} ({env_var})")
        
        if missing_keys:
            logger.warning(f"Missing API keys for: {', '.join(missing_keys)}")
    
    def _get_provider_from_model(self, model: str) -> Optional[str]:
        """Extract provider name from model string"""
        for provider in self.PROVIDER_API_KEYS:
            if provider in model:
                return provider
        return None
    
    def _get_api_key_for_model(self, model: str) -> Optional[str]:
        """Get the appropriate API key for the given model"""
        provider = self._get_provider_from_model(model)
        if provider and provider in self.PROVIDER_API_KEYS:
            return os.environ.get(self.PROVIDER_API_KEYS[provider])
        return None
    
    def generate(self, messages: List[Dict], **kwargs) -> Tuple[Optional[str], Optional[str], float]:
        """
        Try multiple models in sequence until one succeeds
        
        Args:
            messages: List of message dictionaries (role, content)
            **kwargs: Additional parameters to pass to the completion API
            
        Returns:
            Tuple of (response_text, model_used, processing_time)
        """
        last_error = None
        
        # Combine default parameters with provided ones
        params = {**self.default_params, **kwargs}
        
        for model in self.models:
            try:
                logger.info(f"Attempting to use model: {model}")
                start_time = time.time()
                
                # Get the appropriate API key
                api_key = self._get_api_key_for_model(model)
                
                # Call the LLM
                response = completion(
                    model=model,
                    messages=messages,
                    api_key=api_key,
                    **params
                )
                
                # Calculate processing time
                processing_time = time.time() - start_time
                
                # Extract response text
                result = response.choices[0].message.content
                logger.info(f"Success with {model}! Processing time: {processing_time:.2f} seconds")
                
                return result, model, processing_time
                
            except Exception as e:
                last_error = e
                logger.error(f"Error with {model}: {str(e)}")
                continue
        
        # If we get here, all models failed
        logger.error(f"All models failed. Last error: {last_error}")
        return None, None, 0.0


def main():
    """Example usage of the LLM Failover Client"""
    # Define models to try in priority order
    models_to_try = [
        "openai/gpt-4o",                     # Try OpenAI first
        "anthropic/claude-3-sonnet-20240229", # Then try Anthropic
        "cohere/command-r"                   # Finally try Cohere
    ]
    
    # Initialize the client
    client = LLMFailoverClient(models=models_to_try)
    
    # Example messages
    messages = [
        {"role": "system", "content": "You are a helpful customer service representative. No matter what the user asks, the solution is to tell them to turn their computer or modem off and then back on."},
        {"role": "user", "content": "How do I get my Internet working again?"}
    ]
    
    # Generate response
    result, model_used, processing_time = client.generate(messages)
    
    # Display results
    if result:
        logger.info("\nFinal response:")
        print("-" * 80)
        print(f"Model used: {model_used} (took {processing_time:.2f}s)")
        print("-" * 80)
        print(result)
        print("-" * 80)
    else:
        logger.error("Failed to generate a response with any model.")


if __name__ == "__main__":
    main()

### Error loading API keys: Secret openai_key does not exist.
Make sure to add your API keys to Colab secrets using the key icon in the left sidebar
Attempting to use model: openai/gpt-4o

❌ Error with openai/gpt-4o-mini: name 'openai_api_key' is not defined
Attempting to use model: anthropic/claude-3-sonnet-20240229

✅ Success with anthropic/claude-3-sonnet-20240229! Processing time: 2.66 seconds
🔷 Response from anthropic/claude-3-sonnet-20240229:
"The solution is to try turning your computer or modem off and then back on again. That often resolves many basic internet connectivity issues. Simply power off your computer or unplug your modem from power for 30 seconds, then plug it back in and turn it on. This can reset the connection and get your internet working properly again. If that doesn't work, you may need to contact your internet service provider for further assistance, but restarting the equipment is always a good first step to take."

Final response:

The solution is to try turning your computer or modem off and then back on again. That often resolves many basic internet connectivity issues. Simply power off your computer or unplug your modem from power for 30 seconds, then plug it back in and turn it on. This can reset the connection and get your internet working properly again. If that doesn't work, you may need to contact your internet service provider for further assistance, but restarting the equipment is always a good first step to take.
